## 图的遍历

### 挑战：寻找循环





In [ ]:
# 图5-3的循环图邻接列表
graph = {
    'A':['J'],
    'B':['A','F'],
    'C':['B'],
    'F':['c'],
    'J':[]
}

In [ ]:
# 等价代码
class Graph(): 
    """图类"""
    def __init__(self): 
        self.graph = {}  # 初始化图的邻接列表
    def add_edge(self,u,v): 
        if v:
            point = self.graph.get(u) # 尝试获取结点u
            if point:
                point.append(v)       # 若存在直接添加u-v的边  
            else:
                self.graph[u] = [v]   # 若不存在，则先初始化u结点，然后再添加u-v的边
        else:
            self.graph[u] = list()    # 如果v没有值，添加一个空列表 

In [ ]:
from collections import defaultdict # 引入defaultdict,增加字典的自定义
class Graph(): 
    """图类"""
    def __init__(self): 
        self.graph = defaultdict(list) #  初始化图的邻接列表，自定义字典，默认值是列表
    
    def add_edge(self,u,v):
        if v:
            self.graph[u].append(v) # 在u的关键字列表上添加连上v的边
        else:
            self.graph[u] = list()  # 如果v没有值，添加一个空列表

In [ ]:
class GraphCycle(Graph):
    """继承Graph,这是为了检验是否包含循环"""
    def is_cyclic_tool(self, v, visited, recur_stack): 
        visited[v] = True       # 当前结点已访问
        recur_stack[v] = True  # 当前结点放进递归栈中
        # 深度优先遍历每一个邻居结点，如果发现邻居是已经访问， 
        for neighbour in self.graph[v]:
            if visited[neighbour] == False: # 如果结点没有访问，进入该结点
                if self.is_cyclic_tool(neighbour, visited, recur_stack) == True: 
                    return True  # 该结点上发现循环
            elif recur_stack[neighbour] == True: 
                return True      # 该结点已访问，并且也在递归栈中，说明找到循环
        recur_stack[v] = False # 该结点深度遍历完成，移出递归栈
        return False

    def is_cyclic(self):
        # 如果有循环，回复True,否则回复False
        visited = {}   # 初始化参数是否已经访问
        recur_stack = {} # 初始化参数是否在递归栈中
        for key in self.graph.keys():
            visited[key] = False     # 值为未访问状态
            recur_stack[key] = False  # 不在递归栈中
        for node in self.graph.keys(): # 遍历所有结点 
            if visited[node] == False: # 如果结点没有访问，进入该结点
                if self.is_cyclic_tool(node, visited, recur_stack) == True: 
                    return True        # 如果发现有循环，则可以马上返回True
        return False  # 遍历结束后，没有找到循环，则返回False

In [ ]:
g = GraphCycle() 
g.add_edge('A', 'J') 
g.add_edge('B', 'A') 
g.add_edge('B', 'F') 
g.add_edge('C', 'B') 
g.add_edge('F', 'C')
g.add_edge('J', None)
print(g.graph)
g.is_cyclic()

In [ ]:
# 反例-不存在循环
g = GraphCycle() 
g.add_edge('A', 'J') 
g.add_edge('B', 'A') 
g.add_edge('B', 'F') 
g.add_edge('B', 'C') 
g.add_edge('C', 'F')
g.add_edge('F', None)
g.add_edge('J', None)
print(g.graph)
g.is_cyclic()

## 最小生成树

### 克鲁斯卡尔算法

In [ ]:
class GraphPower(): 
    """有权图类"""
    def __init__(self, points):
        self.amount = len(points) # 记录结点的总数
        self.points = points      # 记录结点位置和值的关系
        self.graph = []           #  初始化图的邻接列表
    def add_edge(self, u, v, w):
        if u in self.points and v in self.points:
            index_u = self.points.index(u)
            index_v = self.points.index(v)
            self.graph.append([index_u, index_v, w]) # 录入数据
        else:
            print("录入数据有误")

In [ ]:
class GraphKruskal(GraphPower): 
    """克鲁斯卡尔算法,输入的是有权无向图，求解最小生成树"""
    def find(self, parent, i):
        # 寻找其父结点
        if parent[i] == i: 
            return i 
        return self.find(parent, parent[i]) 
   
    def kruskalMST(self):
        # 基于不相交集实现克鲁斯卡尔算法主程序
        # 第一步初始化
        MST =[] # 初始化MST，也是最终结果 
        parent = [] # 初始化两个列表，用于检测是否有循环,记录结点的父结点
        for node in range(self.amount): # 每一个结点创建一个子集合 
            parent.append(node) 
        index_sorted_edge = 0 # 根据权重已排序的边的下标 
        index_reslut = 0 # MST列表中的下标 
        # 第二步，根据权重排序
        self.graph = sorted(self.graph,key=lambda item: item[2]) # 权重w在item中的三个位置
        while len(MST) < self.amount -1 : # 结束条件：直到生成树中有（N-1）个边 
            # 第三步，选择最小权重的边
            u,v,w = self.graph[index_sorted_edge] 
            index_sorted_edge = index_sorted_edge + 1
            # 检查它是否与MST形成一个循环图。如果没有形成循环图，则把该边放进MST，否则将其丢弃
            parent1 = self.find(parent, u) 
            parent2 = self.find(parent ,v)
            if parent1 != parent2:
                MST.append([u,v,w])         # 结果中添加新的边
                parent[parent2] = parent1  # 更新不相交集
        self.print_result(MST)

    def print_result(self, result):
        print("输出最小生成树结果")
        total = 0
        for u,v,weight in result:
            total += weight
            print ("{} -- {} == {}".format(self.points[u], self.points[v],weight))
        else:
            print("权重总和为：%d" % total)

In [ ]:
g = GraphKruskal(["广州","厦门","成都","武汉","上海","北京"]) 
g.add_edge("广州","厦门", 2)
g.add_edge("广州","武汉", 3) 
g.add_edge("广州","成都", 2) 
g.add_edge("武汉","厦门", 4) 
g.add_edge("武汉","成都", 8) 
g.add_edge("武汉","上海", 1) 
g.add_edge("武汉","北京", 9)
g.add_edge("厦门","上海", 5)
g.add_edge("成都","北京", 7)
g.add_edge("上海","北京", 6)
g.kruskalMST()

### 普里姆算法

In [ ]:
class GraphArray(): 
    """用邻接矩阵表示图"""
    def __init__(self, points):
        self.amount = len(points) # 记录结点的总数
        self.points = points      # 记录结点位置和值的关系
        # 初始化图的邻接矩阵
        self.graph = [[0 for _ in range(self.amount)] 
                    for _ in range(self.amount)]

In [ ]:
class GraphPrimMST(GraphArray): 
    """普里姆算法法,输入的是有权无向图，求解最小生成树"""
    def printMST(self, result): 
        print("边 \t\t 权重")  # 结果输出
        total = 0
        for i in range(1, self.amount):
            total +=  self.graph[i][ result[i]]
            print(self.points[result[i]], "-", self.points[i], "\t", self.graph[i][ result[i]])
        print("总权重和：", total)

    def min_key(self, key, mst):
        # 寻找最小键值的结点下标
        min = float("Inf")  # 默认是无穷大
        for v in range(self.amount): 
            if key[v] < min and mst[v] == False: 
                min = key[v] 
                min_index = v 
        return min_index 

    def primMST(self):
        # 普里姆算法主程序
        key = [float("Inf")] * self.amount # 默认结点键值是无穷大
        parent = [None] * self.amount # 记录最小生成树集合的边，也就是答案
        key[0] = 0  # 把第一个结点作为第一个选择的结点，键值设置为0 
        MST = [False] * self.amount  # 记录最小生成树集合已访问结点
        parent[0] = -1 # 根节点没有父结点，初始化为-1
        for _ in range(self.amount):
            u = self.min_key(key, MST)  # 选择一个在MST中不存在且具有最小键值的结点u
            MST[u] = True # 记录为已访问结点
            for v in range(self.amount):
                # 当边的权重为正，而且没有被访问，若键值比原来小，则更新结点的键值
                if self.graph[u][v] > 0 and MST[v] == False and key[v] > self.graph[u][v]: 
                    key[v] = self.graph[u][v]  # 更新结点键值
                    parent[v] = u  # 更新所选择的边
        self.printMST(parent)

In [ ]:
g = GraphPrimMST(["广州","厦门","成都","武汉","上海","北京"]) 
g.graph = [ [0, 2, 2, 3, 0, 0], 
            [2, 0, 0, 4, 5, 0],
            [2, 0, 0, 8, 0, 7],
            [3, 4, 8, 0, 1, 9],
            [0, 5, 0, 1, 0, 6],
            [0, 0, 7, 9, 6, 0]]
g.primMST()

## 拓扑排序


In [ ]:
class GraphTopological(Graph):
    """解决拓扑排序问题"""
    def topological_sort_util(self, v, visited, stack): 
        visited[v] = True       # 该结点变为已访问
        for i in self.graph[v]: 
            if visited[i] == False: # 结点未访问递归调用函数
                self.topological_sort_util(i, visited, stack) 
        # 相邻结点都访问结束后，把该结点放到栈中
        stack.insert(0,v)  # 把新入栈元素放在表头

    def topological_sort(self):
        # 拓扑排序主程序
        visited = {}   # 初始化参数是否已经访问
        stack = [] # 初始化参数，用列表表示临时栈为空
        for key in self.graph.keys():
            visited[key] = False     # 值为未访问状态
        for node in self.graph.keys(): # 遍历所有结点 
            if visited[node] == False: # 结点是否已经访问
                self.topological_sort_util(node, visited, stack) # 递归进入结点
        print(stack) #把栈保存结果输出

In [ ]:
g = GraphTopological() 
g.add_edge(0, 1) 
g.add_edge(0, 2) 
g.add_edge(0, 3) 
g.add_edge(1, 4) 
g.add_edge(2, 5) 
g.add_edge(3, 6)
g.add_edge(4, 7)
g.add_edge(5, 9)
g.add_edge(6, 7)
g.add_edge(7, 8)
g.add_edge(8, None)
g.add_edge(9, 8)
g.topological_sort() 

In [ ]:
g = TopologicalGraph() 
g.add_edge(0, 2) 
g.add_edge(0, 3)
g.add_edge(0, 1) 
g.add_edge(1, 4) 
g.add_edge(2, 5) 
g.add_edge(3, 6)
g.add_edge(4, 7)
g.add_edge(5, 9)
g.add_edge(6, 7)
g.add_edge(7, 8)
g.add_edge(8, None)
g.add_edge(9, 8)
g.topological_sort() 

### 戴克斯特拉算法（Dijkstra’s algorithm）


In [ ]:
import sys  # 引入系统参数，获取系统支撑的最大值
class GraphDijkstra(GraphArray):
    """这是寻找单源结点到其余结点的最短路径"""
    def print_result(self, dist): 
        print("结点 \t距离源结点的距离")
        for v in range(self.amount): 
            print(self.points[v], "\t", dist[v])

    def min_distance(self, dist, short_path_set):
        # 寻找结点到源结点最短距离
        min_dist = sys.maxsize #初始化最小距离为系统最大值
        for v in range(self.amount):
            # 寻找最小距离的结点，并且不在最短路径树集合中
            if dist[v] <= min_dist and v not in short_path_set: 
                min_dist = dist[v] 
                min_index = v 
        return min_index 

    def dijkstra(self, source):
        # dijkstra算法主程序，遍历所有结点，放进short_path_set集合中，求得所有结点到源结点的最小距离
        distance = [sys.maxsize] * self.amount 
        distance[source] = 0
        short_path_set = set() # 初始化集合为空
        for _ in range(self.amount):
            # 寻找最小距离的结点，并且不包含在最短路径树集合中
            # 而且源结点总是作为第一个结点进入集合
            u = self.min_distance(distance, short_path_set)
            short_path_set.add(u) # 把结点添加到集合中
            # 重新遍历邻接结点，更新结点间最小距离的值
            for v in range(self.amount): 
                if self.graph[u][v] > 0 and v not in short_path_set and distance[v] > distance[u] + self.graph[u][v]: 
                        distance[v] = distance[u] + self.graph[u][v] #符合条件，更新最短路径值
        self.print_result(distance) # 输出结果

In [ ]:
# Driver program 
g = GraphDijkstra(['A','B','C','D','E','F','H','M','J']) 
g.graph = [[0, 0, 0, 4, 6, 0, 0, 0, 0], 
        [0, 0, 9, 8, 0, 0, 1, 7, 0], 
        [0, 9, 0, 0, 0, 2, 0, 11, 10], 
        [4, 8, 0, 0, 0, 0, 13, 0, 0],
        [6, 0, 0, 0, 0, 4, 0, 0, 0], 
        [0, 0, 2, 0, 4, 0, 5, 0, 0], 
        [0, 1, 0, 13, 0, 5, 0, 0, 0], 
        [0, 7, 11, 0, 0, 0, 0, 0, 3], 
        [0, 0, 10, 0, 0, 0, 0, 3, 0] 
        ]; 
g.dijkstra(0) # A作为源结点

In [ ]:
# Driver program 
g = DijkstraGraph(['A','B','C','D']) 
g.graph = [[0, 1, 2, 10], 
        [1, 0, 1, -20], 
        [2, 1, 0, 0], 
        [10, -20, 0, 0],
        ]; 
g.dijkstra(0)# A作为源结点

### 贝尔曼-福特（Bellman–Ford）算法 

In [ ]:
class GraphBellmanFord(GraphPower):
    """贝尔曼-福特（Bellman–Ford）算法"""
    def print_result(self, dist): 
        print("结点到源结点的距离：") 
        for i in range(self.amount): 
            print("{} \t {}".format(self.points[i], dist[i])) 
    
    def bellman_ford(self, source):
        """主程序贝尔曼福特算法求每个结点到源结点最短路径，并且检查是否有负循环"""
        # 第一步：初始化参数，所有结点到源结点的距离为正无穷大 
        distance= [float("Inf")]*self.amount # float("Inf")为正无穷大
        distance[source] = 0  # 源结点本身距离为0
        # 第二步: 进行N-1次的边松弛，找结点到源结点的最短距离
        for i in range(self.amount - 1):
            for u, v, w in self.graph:
                # distance(a) +weight(ab)) < distance(b) 说明存在有更短路径从a到b。
                if distance[u] != float("Inf") and distance[u] + w < distance[v]: 
                        distance[v] = distance[u] + w # 更新最短路径
        # 第三步: 检查是否存在负循环,在完成这么N-1次松弛后如果还是可以松弛（找到更短路径）的话说明存在负循环
        for u, v, w in self.graph:
            if distance[u] != float("Inf") and distance[u] + w < distance[v]: 
                    print("图包含负循环")
                    return
        self.print_result(distance) # 打印结果

In [ ]:
g = GraphBellmanFord(['A','B','C','D','E']) # 初始化图，记录结点值
g.add_edge('A','B', -1) # 添加边
g.add_edge('A','D', 3) 
g.add_edge('B','D', 2)
g.add_edge('B','C', 4)
g.add_edge('B','E', 4)
g.add_edge('C','E', -2)
g.add_edge('E','B', 1)
g.add_edge('E','D', 5)
g.bellman_ford(0) # 按结点A为源结点计算结点的最短距离


In [ ]:
g = BellmanFordGraph(['A','B','C','D','E']) # 初始化图，记录结点值
g.add_edge('A','B', -1) # 添加边
g.add_edge('A','D', 3) 
g.add_edge('B','D', 2)
g.add_edge('B','C', 4)
g.add_edge('B','E', 4)
g.add_edge('C','E', -8)
g.add_edge('E','B', 1)
g.add_edge('E','D', 5)
g.bellman_ford(0) # 按结点A为源结点计算结点的最短距离


### 最大流问题

### 福德-富克逊（Ford-Fulkerson）算法

In [ ]:
class GraphFordFulkerson(GraphArray): 
    def BFS(self, s, t, parent):
        """广度优先搜索"""
        visited =[False]*(self.amount) # 初始化所有结点都没有访问过
        queue=[] # 用队列结构来记录访问历史
        queue.append(s) # 首先访问起点s
        visited[s] = True
        while len(queue) > 0:   # 当队列为空，说明全部顶点已经遍历完成 
            current_point = queue.pop(0)  # 获取访问历史的队头为当前顶点
            for ind, val in enumerate(self.graph[current_point]):
                # 逐一访问当前顶点的所有关联点
                if visited[ind] == False and val > 0:
                    queue.append(ind) # 如果没有访问添加到已访问队列中 
                    visited[ind] = True  # 标记结点为已访问
                    parent[ind] = current_point   # 记录遍历的路径链接
        return True if visited[t] else False  # 如果有路径能到终点t则返回True
            
    def FordFulkerson(self, source, sink): 
        # 福德-富克逊算法，计算从起点到终点的最大流问题 
        parent = [-1]*(self.amount) # 这个列表记录路径链接
        max_flow = 0 # 初始化最大总流量为0 
        while self.BFS(source, sink, parent):  # 通过BFS来寻找增广路径
            path_flow = float("Inf") # 路径上的流量，初始化为无穷大
            s = sink 
            while(s != source):
                # 寻找此增广路径上的最小流量 
                path_flow = min(path_flow, self.graph[parent[s]][s]) 
                s = parent[s]
            max_flow += path_flow # 把此增广路径上的流量加到总流量上
            # 更新此增广路径上的残余容量
            v = sink 
            while(v != source): 
                u = parent[v]
                self.graph[u][v] -= path_flow  # 正向边减去路径流量
                self.graph[v][u] += path_flow  # 反向边加上路径流量
                v = parent[v]
        print("总的最大流量为：",max_flow)
        return max_flow

In [ ]:
g = GraphFordFulkerson(['s','1','2','3','t']) 
g.graph = [[0, 4, 0, 7, 0], 
        [0, 0, 3, 2, 0], 
        [0, 0, 0, 0, 5], 
        [0, 0, 0, 0, 4], 
        [0, 0, 0, 0, 0]] 
source = 0 # 起点下标
sink = 4 # 终点下标
g.FordFulkerson(source, sink)

### 挑战：朋友圈扩列

In [ ]:
class GraphFriend(): 
    def __init__(self, points):
        self.amount = len(points) # 记录结点的总数
        self.points = points      # 记录结点位置和值的关系
        self.group = [[] for i in range(self.amount)] # 无向图，保存所有人的人脉关系
        self.groups = [] # 记录全部朋友圈子的情况
    
    def add_relation(self, u, v):
        # 添加朋友关系
        if u in self.points and v in self.points:
            index_u = self.points.index(u)
            index_v = self.points.index(v)
            self.group[index_u].append(index_v) # 无向图，两边互加
            self.group[index_v].append(index_u) # 无向图，两边互加
        else:
            print("录入数据有误")

    def _count_util(self, v, visited):
        # 深度优先搜索发现朋友圈子，参考循环图小节程序
        visited[v] = True # 记录结点已经访问
        i = 0
        while i != len(self.group[v]): # 遍历所有的边
            if (not visited[self.group[v][i]]): # 把所有能访问的结点都访问
                self._count_util(self.group[v][i], visited) 
            i += 1
        return
    
    def count_groups(self):
        # 统计朋友圈子主程序
        visited = [0] * self.amount # 初始化所有结点未被访问
        has_group = [] # 保存已经归类的人
        for i in range(self.amount):
            if (visited[i] == False): # 如果一个结点没有被访问过，就开始一个朋友圈子
                self._count_util(i, visited)
                g = [] # 记录当前圈子的所有人
                for vi,val in enumerate(visited):
                    if val: # 如果是True,说明结点已经访问了
                        if vi in has_group: # 如果此人已经归类了，跳过不处理
                            continue
                        else:
                            g.append(vi) # 添加此人都这个圈子
                            has_group.append(vi) # 保存为已归类的人
                self.groups.append(g) # 保存圈子结果
        # 输出结果
        print("存在有%d个朋友圈子" % len(self.groups))
        print("每个圈子情况：")
        for i, group in enumerate(self.groups):
            print("第%d圈子:" % (i+1), "-".join([self.points[p] for p in group]))
        
    
    def _count_degree(self, person, person2, visited):
        visited[person2] = True # 记录结点已经访问
        # 深度优先探测人脉度
        if person in self.group[person2]: # 如果在好友中，那么返回人脉度
            return [[person2]]
        result = [] # 保存所有人脉路径
        for p in self.group[person2]:
            if not visited[p]:
                for path in self._count_degree(person, p, visited):
                    result.append([person2] + path)
        return result
    
    def relation_degree(self, person1, person2):
        # 求出两人之间的人脉度，如果互相为好友，则为1，多间隔一个朋友人脉度则多加1
        visited = [0] * self.amount # 初始化所有结点未被访问
        if person1 not in self.points or person2 not in self.points:
            print("数据输入有误")  # 查看输入的结点是否存在，若不存在结束程序
            return
        p1_index = self.points.index(person1)
        p2_index = self.points.index(person2)
        if p1_index == p2_index:  # 若是相同的一个人，也不参与运算，马上结束程序
            print("这是同一个人，请输入不同的两个人")
            return
        for group in self.groups:
            # 先查看两人是否在一个圈子里面
            if p1_index in group and p2_index in group: # 在同一个圈子才有人脉路径
                visited[p1_index] = True # 不需要寻找自己本身
                visited[p2_index] = True # 不需要寻找自己本身
                for path in self._count_degree(p1_index, p2_index, visited):
                    path.append(p1_index) # 补充开始结点到结果中
                    links = path[::-1] # 翻转列表，让结果更容易理解
                    print("{}和{}的人脉为{}度".format(person1, person2, len(links)))
                    print("-".join([self.points[p] for p in links]))
                break
        else:
            print("{}和{}两人不在一个朋友圈子".format(person1, person2))            

In [ ]:
g = GraphFriend(['A','B','C','D','E','F','G']) #  初始化图
g.add_relation('A', 'B') # 'A'和‘B’是朋友
g.add_relation('A', 'E') # 'A'和‘E’是朋友
g.add_relation('B', 'C')
g.add_relation('B', 'D')
g.add_relation('C', 'E')
g.add_relation('F', 'G')
g.count_groups() # 这群用户中有多少个群呢？
print('---------人脉探索---------------')
g.relation_degree('A', 'C')
print('---------E-D人脉探索------------')
g.relation_degree('E', 'D')
print('---------G-E人脉探索------------')
g.relation_degree('G', 'E')

In [ ]:
class GraphDegree(GraphFriend):
    # 继承上面的类，添加计算最长人脉度的函数
    def DFS(self, src_person, prev_degree, max_len, visited): 
        visited[src_person] = 1 # 标记为已访问
        curr_degree = 0  # 记录当前人脉度大小
        friend = None    # 好友,也就是邻接的结点
        for p in self.group[src_person]:# 遍历所有好友
            if not visited[p]: # 好友未被访问
                curr_degree = prev_degree + 1
                # 深度优先搜索探索人脉度
                self.DFS(p, curr_degree, max_len, visited)
            # 对比，保存当前最长的人脉度
            if (max_len[0] < curr_degree): 
                max_len[0] = curr_degree 
            curr_degree = 0 # 重新初始化人脉度

    def longest_degree(self):
        # 求解最长人脉度主程序
        max_len = [-999999999999] # 初始化结果
        for i in range(0, self.amount):
            visited = [False] * (self.amount) # 初始化所有好友未被访问 
            self.DFS(i, 0, max_len, visited) # 探索人脉路径
        return max_len[0]

In [ ]:
g = GraphDegree(['A','B','C','D','E','F','G']) #  初始化图
g.add_relation('A', 'B') # 'A'和‘B’是朋友
g.add_relation('A', 'E') # 'A'和‘E’是朋友
g.add_relation('B', 'C')
g.add_relation('B', 'D')
g.add_relation('C', 'E')
g.add_relation('F', 'G')
print(g.longest_degree())
g.count_groups() # 这群用户中有多少个群呢？
g.relation_degree('E', 'D')